# SWC Simulator Library Workflow Example

### Create a simple neuron and run it $M$ times to generate data; then save it.

In [ ]:
from compact_dependencies import *
from swcsim import *
from model import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from StringIO import StringIO
import re

# a simple neuron in SWC format
# 3 - 2
#       > 1 - 0
# 5 - 4
swcString = \
"id type x y z radius parent_id\n" \
"0  2    3 1 0 1      -1\n" \
"1  1    2 1 0 1       0\n" \
"2  1    1 2 0 1       1\n" \
"3  1    0 2 0 1       2\n" \
"4  1    1 0 0 1       1\n" \
"5  1    0 0 0 1       4\n"
dataString = StringIO(re.sub(' +',' ',swcString))

neuron = SWCNeuron(dataString, swcKwargs={'delimiter':' '})

dt = 1e-5
dur = 0.5
t = np.arange(0,dur,dt)
stimulus = np.zeros_like(t)
M = 100

V = np.zeros((M, neuron.graph.number_of_nodes(), len(t)))
I = np.zeros((M, neuron.graph.number_of_edges(), len(t)))
for m in range(M):
    if m % round(M/10) == 0:
        print(str(float(m)/M) + "%")
    stimulus = 2000.0 * np.random.poisson(lam=0.25, size=t.shape)

    # Create a simulator
    sim = neuron.create_simulator(dt, [3,5], stimulus, steps=len(stimulus))

    # Run the simulator

    for i, _ in enumerate(sim):
        for j,n in enumerate(neuron.graph.nodes_iter()):
            V[m,j,i] = neuron.graph.node[n]['states'].V
        for j, (u,v,d) in enumerate(neuron.graph.edges_iter(data=True)):
            I[m,j,i] = d['I']
            
save_large_dataset('I',I)
save_large_dataset('V',V)

### Use a simple RNN model to try to recover the dynamics of the neuron.

In [ ]:
from compact_dependencies import *
from neuron_compressor import *
batch_size = 32
epochs = 200
N = 6

I = load_large_dataset('I')
V = load_large_dataset('V')
I = I / np.max(I)
V = V / np.max(V)

print('Beginning evaluation...')
model = neuron_compressor(simplegru_model(N, I, V), batch_size = batch_size, epochs = epochs)

Using Theano backend.
Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


Beginning evaluation...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50000L, 6)         288       
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________
Train on 75 samples, validate on 25 samples
Epoch 1/50
75/75 [==============================] - 72s - loss: 0.0838 - val_loss: 0.0809
Epoch 2/50
75/75 [==============================] - 71s - loss: 0.0809 - val_loss: 0.0782
Epoch 3/50
64/75 [========================>.....] - ETA: 7s - loss: 0.0783 